In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").getOrCreate()

print(spark.sparkContext.version)

3.3.2


##### create sample df

In [0]:
data = [("ABC", "Q1", 6000),("ABC", "Q2", 8000),("ABC", "Q3", 4000),("ABC", "Q4", 7000),
        ("XYZ", "Q1",2000),("XYZ", "Q2",4000),("XYZ", "Q3",1000),("XYZ", "Q4",5000)]
schema = ["company_name" , "quarter" , "revenue" ]

df = spark.createDataFrame(data = data, schema = schema)
display(df)

company_name,quarter,revenue
ABC,Q1,6000
ABC,Q2,8000
ABC,Q3,4000
ABC,Q4,7000
XYZ,Q1,2000
XYZ,Q2,4000
XYZ,Q3,1000
XYZ,Q4,5000


##### pivot from above df

In [0]:
pivot_df = df.groupBy("company_name").pivot("quarter").sum("revenue")
display(pivot_df)

company_name,Q1,Q2,Q3,Q4
XYZ,2000,4000,1000,5000
ABC,6000,8000,4000,7000


##### un pivoting

In [0]:
unpivot_df = pivot_df.selectExpr("company_name", "stack(4,'Q1',Q1,'Q2',Q2,'Q3',Q3,'Q4',Q4)as(quarter_name,revenue)")
display(unpivot_df)


company_name,quarter_name,revenue
XYZ,Q1,2000
XYZ,Q2,4000
XYZ,Q3,1000
XYZ,Q4,5000
ABC,Q1,6000
ABC,Q2,8000
ABC,Q3,4000
ABC,Q4,7000


In [0]:
from pyspark.sql import functions as F
data1 = [("satish", "M", 6000),("swapna", "F", 8000),("suresh", "M", 4000),("srivani", "F", 7000),
        ("srikanth", "M",2000),("divya", "F",4000),("srinivas", "M",1000),("swaroopa", "F",5000)]
schema1 = ["name" , "gender" , "salary" ]

df1 = spark.createDataFrame(data = data1, schema = schema1)
display(df1)
female_sal = df1.filter(df1.gender == 'F').agg(F.sum(df1.salary).alias("sumoffemalesal"))
display(female_sal)

name,gender,salary
satish,M,6000
swapna,F,8000
suresh,M,4000
srivani,F,7000
srikanth,M,2000
divya,F,4000
srinivas,M,1000
swaroopa,F,5000


sumoffemalesal
24000


In [0]:
pivot_df1 = df1.groupBy("name").pivot("gender").sum("salary")
display(pivot_df1)
unpivot_df1 = pivot_df1.selectExpr("name", "stack(2,'F',F,'M',M) as (gender,salary)").where("salary is not null")
display(unpivot_df1)


name,F,M
swaroopa,5000,null
srinivas,null,1000
srikanth,null,2000
suresh,null,4000
swapna,8000,null
divya,4000,null
satish,null,6000
srivani,7000,null


name,gender,salary
swaroopa,F,5000
srinivas,M,1000
srikanth,M,2000
suresh,M,4000
swapna,F,8000
divya,F,4000
satish,M,6000
srivani,F,7000
